In [1]:
from tqdm import tqdm
import math
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import json
import re
import spacy
sp = spacy.load('en_core_web_sm')

In [2]:
f = open('./labelled_sentences.json')
data = json.load(f)
f.close

<function TextIOWrapper.close()>

In [3]:
data[0]['Label']['classifications'][0]['answer'][0]['value']

'negative'

In [4]:
data[0]['Labeled Data']

'Other cities far from the epicentre have restricted the movement of residents, with a 14-day self-quarantine for people returning to Beijing.'

In [5]:
punctuation = re.compile('[^\w\s]')
space = re.compile('  +')

clean_data = []
for i in range(len(data)):
    if bool(data[i]['Label']):


        sentence = sp(space.sub(' ', punctuation.sub('', data[i]['Labeled Data'])))
        




        if data[i]['Label']['classifications'][0]['answer'][0]['value'] == 'neutral':
            clean_data.append({'sentence': sentence, 'label': 0})
        elif data[i]['Label']['classifications'][0]['answer'][0]['value'] == 'negative':
            clean_data.append({'sentence': sentence, 'label': -1})
        else:
            clean_data.append({'sentence': sentence, 'label': 1})
    
clean_data[0]

{'sentence': Other cities far from the epicentre have restricted the movement of residents with a 14day selfquarantine for people returning to Beijing,
 'label': -1}

In [6]:
sentences_lemmaed = []
for sentence in clean_data:
    words = []
    for w in sentence['sentence']:
        words.append(w.lemma_.lower())
    sentences_lemmaed.append([" ".join(words), sentence['label']])
sentences_lemmaed[2]

['after all the newsletter -pron- have run for over a decade motley fool stock advisor have triple the market',
 0]

In [7]:
words = set()

for par in tqdm(sentences_lemmaed):
    for w in par[0].split():
        words.add(w)

words = list(words)
        
len(words)

100%|██████████| 1552/1552 [00:00<00:00, 125771.58it/s]


6474

In [ ]:
class_sentence_count = {-1: 0, 0: 0, 1: 0}

for par in tqdm(sentences_lemmaed):
    class_paragraph_count[par[1]] += 1

In [8]:
word_class_sentence_count = {}

for word in tqdm(words):
    word_class_sentence_count[word] = {-1: 0, 0: 0, 1: 0}
    for par in sentences_lemmaed:
        if word in par[0].split():
            word_class_sentence_count[word][par[1]] += 1


100%|██████████| 6474/6474 [00:22<00:00, 292.17it/s]


In [9]:
word_sentence_count = {}
for k, v in word_class_sentence_count.items():
    word_sentence_count[k] = sum(v.values())

In [10]:
total = len(sentences_lemmaed)
total

1552

In [11]:
pmi = {}

# p(w|c)
for word in tqdm(words):
    pmi[word] = {}
    
    if word_class_sentence_count[word][-1] != 0:
        pmi[word][-1] = math.log(
            (word_class_sentence_count[word][-1] / total) / 
            (class_sentence_count[-1] / total))
    else:
        pmi[word][-1] = math.log(
            (1 / total) / 
            (class_sentence_count[-1] / total))


    if word_class_sentence_count[word][0] != 0:
        pmi[word][0] = math.log(
            (word_class_sentence_count[word][0] / total) / 
            (class_sentence_count[0] / total))
    else:
        pmi[word][0] = math.log(
            (1 / total) / 
            (class_sentence_count[0] / total))

    
        
    if word_class_sentence_count[word][1] != 0:
        pmi[word][1] = math.log(
            (word_class_sentence_count[word][1] / total) / 
            (class_sentence_count[1] / total))
    else:
        pmi[word][1] = math.log(
            (1 / total) / 
            (class_sentence_count[1] / total))

  0%|          | 0/6474 [00:00<?, ?it/s]


NameError: name 'class_sentence_count' is not defined

In [47]:
S = {}

for word, values in pmi.items():
    S[word] = values[1] - values[-1]

In [51]:
s_df = pd.DataFrame.from_dict(S, orient='index')
s_df.columns = ['sent']
freq = pd.DataFrame.from_dict(word_sentence_count, orient='index')
freq.columns = ['freq']
s_df = s_df.join(freq)
s_df.sort_values('sent', ascending=False)

,sent,freq
ensure,3.061468,14
small,2.966158,15
thank,2.455333,6
dose,2.455333,7
research,2.455333,15
...,...,...
down,-2.010576,45
confirm,-2.044477,35
concern,-2.109016,23
resident,-2.109016,29


In [52]:
s_df.loc['negative']

sent    0.663573
freq    3.000000
Name: negative, dtype: float64

In [58]:
vader_lexicon_file = './vader_lexicon.txt'
vader_lexicon = {}
with open(vader_lexicon_file, 'r') as f:
    for line in f.readlines():
        line = line.strip()
        vader_lexicon[line.split('\t')[0]] = float(line.split('\t')[1])
vader_words = list(vader_lexicon.keys())

s_new = s_df[~s_df.index.isin(vader_words)].copy()
s_com = s_df[s_df.index.isin(vader_words)].copy()

s_com['vader'] = s_com.apply(lambda x: vader_lexicon.get(x.name), axis=1)


In [59]:
s_com.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   sent    0 non-null      float64
 1   freq    0 non-null      int64  
 2   vader   0 non-null      float64
dtypes: float64(2), int64(1)
memory usage: 0.0+ bytes


In [60]:
s_com_filter = s_com[s_com['freq']>100].copy().reset_index()

In [63]:
px.scatter(s_com_filter, x='vader', y='sent', text='index', size='freq',
           labels={'vader': 'Vader score', 'sent': 'Customized score'}, 
           title='Comparison of Lexicon')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed